<h3>For Regularly adding new data (features and labels), useful for retraining purposes. </h3>

In [4]:
from helper_functions import synthetic_data_model, patient_id_and_date, clean_up
import hopsworks
from dotenv import load_dotenv
import os

In [2]:
model_dir = "synth_data_model"
model_path = os.path.join(model_dir, "synth_data_model.pkl")

model = synthetic_data_model(model_path)


In [3]:
new_synth_data = model.sample(100)
new_synth_data = clean_up(new_synth_data)
new_synth_data.diabetes.value_counts()

diabetes
0    50
1    50
Name: count, dtype: int64

In [6]:
# conditions = (
#     (new_synth_data['age'] >= 1) & (new_synth_data['age'] <= 120) &
#     (new_synth_data['bmi'] >= 10) & (new_synth_data['bmi'] <= 100) &
#     (new_synth_data['HbA1c_level'] >= 3) & (new_synth_data['HbA1c_level'] <= 15) &
#     (new_synth_data['blood_glucose_level'] >= 50) & (new_synth_data['blood_glucose_level'] <= 400))

# new_synth_data[~conditions]

,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,diabetes


In [5]:
new_synth_data = patient_id_and_date(new_synth_data)
new_synth_data.head(3)

,patient_id,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,diabetes,datetime
0,0,72,0,0,23.77,4.0,85,0,2024-07-18 15:49:55.081271
1,1,79,1,0,41.20,6.5,144,1,2024-07-18 15:49:55.081271
2,2,54,0,0,35.55,6.7,143,1,2024-07-18 15:49:55.081271


In [6]:
load_dotenv()
hopswork_key = os.getenv("HOPSWORK_KEY")

In [7]:
project = hopsworks.login(api_key_value=hopswork_key)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/112801
Connected. Call `.close()` to terminate connection gracefully.


In [8]:
fg = fs.get_or_create_feature_group(
    name="diabetes_features_backfill",
    version=1)

In [9]:
fg.insert(new_synth_data, write_options={"wait_for_job" : False})

2024-07-18 15:50:31,510 WARNING: FeatureGroupWarning: The ingested dataframe contains upper case letters in feature names: `['HbA1c_level']`. Feature names are sanitized to lower case in the feature store.



Uploading Dataframe: 0.00% |          | Rows 0/100 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: diabetes_features_backfill_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/112801/jobs/named/diabetes_features_backfill_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f121ee13910>, None)